In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch.nn.functional as F
import torch
from torch import nn
from torchsummary import summary
from importlib.util import find_spec
if find_spec("text_recognizer") is None:
    import sys
    sys.path.append('..')

from text_recognizer.networks.transformer.vit import ViT
from text_recognizer.networks.transformer.transformer import Transformer
from text_recognizer.networks.transformer.layers import Decoder
from text_recognizer.networks.transformer.nystromer.nystromer import Nystromer

In [1]:
from pathlib import Path

In [2]:
import attr

In [9]:
@attr.s
class B:
    batch_size = attr.ib()
    num_workers = attr.ib()

In [10]:
@attr.s
class T(B):

    def __attrs_post_init__(self) -> None:
        super().__init__(self.batch_size, self.num_workers)
        self.hej = None
    
    batch_size = attr.ib()
    num_workers = attr.ib()
    h: Path = attr.ib(converter=Path)

In [11]:
t = T(batch_size=16, num_workers=2, h="hej")

In [13]:
t.h

PosixPath('hej')

In [12]:
t.batch_size

16

In [11]:
t.h

PosixPath('hej')

In [21]:
for f in Path("../text_recognizer").glob("**/*.py"):
    print(f)

../text_recognizer/__init__.py
../text_recognizer/callbacks/__init__.py
../text_recognizer/callbacks/wandb_callbacks.py
../text_recognizer/data/image_utils.py
../text_recognizer/data/emnist.py
../text_recognizer/data/iam_lines.py
../text_recognizer/data/download_utils.py
../text_recognizer/data/mappings.py
../text_recognizer/data/iam_preprocessor.py
../text_recognizer/data/__init__.py
../text_recognizer/data/make_wordpieces.py
../text_recognizer/data/iam_paragraphs.py
../text_recognizer/data/sentence_generator.py
../text_recognizer/data/emnist_lines.py
../text_recognizer/data/build_transitions.py
../text_recognizer/data/base_dataset.py
../text_recognizer/data/base_data_module.py
../text_recognizer/data/iam.py
../text_recognizer/data/iam_synthetic_paragraphs.py
../text_recognizer/data/transforms.py
../text_recognizer/data/iam_extended_paragraphs.py
../text_recognizer/networks/__init__.py
../text_recognizer/networks/util.py
../text_recognizer/networks/cnn_tranformer.py
../text_recognizer

In [12]:
Path("..").glob("**/*.py")

<generator object Path.glob at 0x7ff8bb9ce5f0>

In [4]:
from text_recognizer.networks.encoders.efficientnet.efficientnet import EfficientNet

In [5]:
en = EfficientNet("b0")

In [ ]:
summary(en, (1, 224, 224));

In [ ]:
torch.cuda.is_available()

In [ ]:
decoder = Decoder(dim=128, depth=2, num_heads=8, ff_kwargs={}, attn_kwargs={}, cross_attend=True)

In [ ]:
decoder.cuda()

In [ ]:
transformer_decoder = Transformer(num_tokens=1003, max_seq_len=451, attn_layers=decoder, emb_dim=128, emb_dropout=0.1)

In [ ]:
transformer_decoder.cuda()

In [ ]:
efficient_transformer = Nystromer(
    dim = 64,
    depth = 4,
    num_heads = 8,
    num_landmarks = 64
)

In [ ]:
v = ViT(
    dim = 64,
    image_size = (576, 640),
    patch_size = (32, 32),
    transformer = efficient_transformer
).cuda()

In [6]:
t = torch.randn(4, 1, 576, 640).cuda()

In [7]:
en.cuda()

EfficientNet(
  (_conv_stem): Sequential(
    (0): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
    (1): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (2): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    (3): Mish(inplace=True)
  )
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
        (2): Mish(inplace=True)
      )
      (_squeeze_excite): Sequential(
        (0): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
        (1): Mish(inplace=True)
        (2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
      )
      (_pointwise): Sequential(
        (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
      )
    )

In [8]:
en(t).shape

torch.Size([4, 1280, 18, 20])

In [ ]:
o = v(t)

In [ ]:
caption = torch.randint(0, 90, (16, 690)).cuda()

In [ ]:
o.shape

In [ ]:
caption.shape

In [ ]:
o = torch.randn(16, 20 * 18, 128).cuda()

In [ ]:
caption = torch.randint(0, 1000, (16, 200)).cuda()

In [ ]:
transformer_decoder(caption, context = o).shape # (1, 1024, 20000)

In [ ]:
efficient_transformer = efficient_transformer(num_landmarks=256)

In [ ]:
efficient_transformer()

In [ ]:
from omegaconf import OmegaConf

In [ ]:
path = "../training/configs/vqvae.yaml"

In [ ]:
conf = OmegaConf.load(path)

In [ ]:
print(OmegaConf.to_yaml(conf))

In [ ]:
from text_recognizer.networks import VQVAE

In [ ]:
vae = VQVAE(**conf.network.args)

In [ ]:
vae

In [ ]:
datum = torch.randn([2, 1, 576, 640])

In [ ]:
vae.encoder(datum)[0].shape

In [ ]:
vae(datum)[0].shape

In [ ]:
datum = torch.randn([2, 1, 576, 640])

In [ ]:
trg = torch.randint(0, 1000, [2, 682])

In [ ]:
trg.shape

In [ ]:
datum = torch.randn([2, 1, 224, 224])

In [ ]:
en(t).shape

In [ ]:
path = "../training/configs/cnn_transformer.yaml"

In [ ]:
conf = OmegaConf.load(path)

In [ ]:
print(OmegaConf.to_yaml(conf))

In [ ]:
from text_recognizer.networks.cnn_transformer import CNNTransformer

In [ ]:
t = CNNTransformer(input_shape=(1, 576, 640), output_shape=(682, 1), **conf.network.args)

In [ ]:
t.encode(datum).shape

In [ ]:
trg.shape

In [ ]:
t(datum, trg).shape

In [ ]:
b, n = 16, 128
device = "cpu"

In [ ]:
x = lambda: torch.ones((b, n), device=device).bool()

In [ ]:
x().shape

In [ ]:
torch.ones((b, n), device=device).bool().shape

In [ ]:
x = torch.randn(1, 1, 576, 640)

In [ ]:
576 // 32

In [ ]:
640 // 32

In [ ]:
18 * 20

In [ ]:
x = torch.randn(1, 1, 144, 160)

In [ ]:
from einops import rearrange

In [ ]:
patch_size=16
p = rearrange(x, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_size, p2 = patch_size)

In [ ]:
p.shape